# Alzheimer

In [1]:
from keras.src.models import Sequential
from keras.src.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image
import tensorflow as tf

In [2]:
# Verificar se há GPUs disponíveis
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print(f"GPUs disponíveis: {gpus}")
else:
    print("Nenhuma GPU disponível.")

Nenhuma GPU disponível.


In [6]:
# Inicializa a CNN
classfier = Sequential()

# Convolução e pooling
classfier.add(Conv2D(32, (3, 3), input_shape=(256, 124, 1), activation='relu')) 
classfier.add(MaxPooling2D(pool_size=(2, 2)))

classfier.add(Conv2D(64, (3, 3), activation='relu'))
classfier.add(MaxPooling2D(pool_size=(2, 2)))

classfier.add(Conv2D(128, (3, 3), activation='relu'))
classfier.add(MaxPooling2D(pool_size=(2, 2)))

classfier.add(Conv2D(256, (3, 3), activation='relu'))
classfier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening
classfier.add(Flatten())

# Conexão completa
classfier.add(Dense(units=128, activation='relu'))
classfier.add(Dense(units=1, activation='sigmoid'))

# Compilando a rede com class_weight
classfier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

In [7]:
# Aumentando dados
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Assegurando que a proporção de classes seja considerada
training_set = train_datagen.flow_from_directory(
    'data/mri/train',
    target_size=(256, 124),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale'
)

test_set = test_datagen.flow_from_directory(
    'data/mri/validation',
    target_size=(256, 124),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale'
)

Found 18656 images belonging to 2 classes.
Found 3292 images belonging to 2 classes.


In [8]:
# Treinamento da rede
classfier.fit(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=5,
    validation_data=test_set,
    validation_steps=len(test_set)
)

Epoch 1/5
583/583 [==============================] - 160s 273ms/step - loss: 0.3664 - accuracy: 0.8340 - precision: 0.8043 - recall: 0.6825 - val_loss: 0.2390 - val_accuracy: 0.8940 - val_precision: 0.8011 - val_recall: 0.9195
Epoch 2/5
583/583 [==============================] - 190s 325ms/step - loss: 0.1885 - accuracy: 0.9241 - precision: 0.8951 - recall: 0.8822 - val_loss: 0.1005 - val_accuracy: 0.9620 - val_precision: 0.9483 - val_recall: 0.9407
Epoch 3/5
583/583 [==============================] - 196s 336ms/step - loss: 0.1226 - accuracy: 0.9524 - precision: 0.9338 - recall: 0.9271 - val_loss: 0.1251 - val_accuracy: 0.9481 - val_precision: 0.8714 - val_recall: 0.9956
Epoch 4/5
583/583 [==============================] - 183s 314ms/step - loss: 0.0877 - accuracy: 0.9664 - precision: 0.9521 - recall: 0.9500 - val_loss: 0.0537 - val_accuracy: 0.9812 - val_precision: 1.0000 - val_recall: 0.9451
Epoch 5/5
583/583 [==============================] - 188s 322ms/step - loss: 0.0637 - accura

In [10]:
# Teste
test_image = image.load_img('data/mri/AD-0-31273-23010.png', color_mode='grayscale', target_size=(256, 124))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0) / 255.0  # Normalização

result = classfier.predict(test_image)
print(result)

if result[0][0] >= 0.5:
    prediction = 'Not Alzheimer'
else:
    prediction = 'Alzheimer'
print(prediction)

1/1 [==============================] - 0s 47ms/step
[[0.00119768]]
Alzheimer
